In [ ]:
utils.sql_query("SELECT * FROM")

In [1]:
import sys
import pandas as pd
import utils
import utils_bux
import featuretools as ft
from sklearn.externals import joblib

/home/jo/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### DEFINE PIPELINE PARAMETERS

In [2]:
load_to_vertica = False

# the timeframe of extracted users
# users_from = '2016-01-01'
# users_till = '2016-01-07'
cohort_size = 1000

# the timeframe of extracted behavioral data
interval = '1 week'

# the type of the prediction problem
# 'regression', 'binary classification', 'multiclass classification'
prediction_problem_type = 'multiclass classification'

# # multiclass values
# medium_value = 5
# high_value = 50

print("Pipeline parameters defined")
print("Extraction of scoring for users from", users_from, "till", users_till)

Pipeline parameters defined


NameError: name 'users_from' is not defined

### CONNECT TO THE DATABASE

In [5]:
# connect to the vertica database, create a cursor
conn, cur = utils.connect_to_db()

Connected to the database


### BUILD ENTITIES

#### Cohorts entity

In [4]:
cohorts = utils_bux.build_cohorts_entity(cur=cur,
                                         users_from=users_from,
                                         users_till=users_till)

Cohorts entity built


#### Users entity

In [5]:
user_details = utils_bux.build_users_entity(cur=cur,
                                            users_from=users_from,
                                            users_till=users_till,
                                            interval=interval,
                                            cohorts=cohorts,
                                            cohort_size=cohort_size)

Users entity built with 7891 users


#### Transactions entity

In [6]:
daily_transactions = utils_bux.build_transactions_entity(cur=cur,
                                                         interval=interval)

Transactions entity built with 63128 transactions


In [5]:
### no need for labels

### CREATE THE ENTITY SET

In [6]:
es = utils_bux.create_bux_entity_set(cohorts, user_details, daily_transactions)
es

NameError: name 'cohorts' is not defined

### FEATURE ENGINEERING (DFS)

In [7]:
top_features = ft.load_features("top_features", es)
fm = utils.calculate_feature_matrix_top_features(es, top_features)
X = fm.reset_index(drop=True)
# X = fm.reset_index().merge(labels)
# X.to_csv("production_features.csv")
print("Features built:\n", list(fm.columns))

NameError: name 'es' is not defined

### LOADING THE MODEL

In [8]:
model = joblib.load('models/model.pkl')
print("Model loaded")

Model loaded


### SCORING

In [18]:
# X = fm.reset_index().merge(labels)
# X, y = utils.make_labels(X, prediction_problem_type)
# X_train, X_test, y_train, y_test = utils.train_test_splitting(X, y)
# model = utils.xgboost_train(X_train, y_train, prediction_problem_type)
# y_pred = utils.xgboost_predict(model, X_test, prediction_problem_type)
y_pred = utils.xgboost_predict(model, X, prediction_problem_type)
print("Prediction done")

Prediction done


### SAVE RESULTS AS A CSV

In [19]:
# utils.sql_query(cur, "SELECT * FROM analytics.model_scoring_predictions LIMIT 10")

In [20]:
# topic_type	report_date	model_type	user_id	class_prediction	prob
predictions = pd.DataFrame()
predictions["user_id"] = user_details["user_id"]
predictions["topic_type"] = "clv_prediction"
predictions['report_date'] = pd.to_datetime('today').strftime("%Y-%m-%d")
predictions["model_type"] = "xgboost"
predictions["class_prediction"] = y_pred
predictions["prob"] = 0
predictions = predictions[["topic_type", "report_date", "model_type", "user_id", "class_prediction", "prob"]]
predictions.head()

,topic_type,report_date,model_type,user_id,class_prediction,prob
0,clv_prediction,2018-05-11,xgboost,0043c230-2ecb-400f-b37c-6c46ac1038a2,0,0
1,clv_prediction,2018-05-11,xgboost,0176f814-293b-40fb-b20c-3c63d62539dc,0,0
2,clv_prediction,2018-05-11,xgboost,019abcc9-aaca-4401-a668-fb6e08e454fc,0,0
3,clv_prediction,2018-05-11,xgboost,0260e2cd-0946-4a41-b65a-cc83ce84c1b3,0,0
4,clv_prediction,2018-05-11,xgboost,02e75d5b-db4d-4ffa-a88d-b93b85330642,0,0


In [7]:
import utils
import pandas as pd
import dask.dataframe as dd
import os
import pandas
import tempfile

In [9]:
# conn, cur = utils.connect_to_db()
os.chdir('/home/jo/Documents/Master thesis @ BUX/notebooks/scoring')
predictions = dd.read_csv('*.csv').compute()
print("Number of users:", len(predictions))

def copy_to_vertica(source_df, destination_table, connection, include_index=False, truncate=False, verbose=False):

    if truncate == True:
        print("Truncating table " + destination_table)
        cur = connection.cursor()
        cur.execute("truncate table " + destination_table)

    columns = list(source_df)
    
    if include_index == True:
        columns.insert(0, source_df.index.name)
    
    columns = " " + str(tuple(source_df)).replace("'","")
        
    tmp_file = tempfile.NamedTemporaryFile()
    source_df.to_csv(tmp_file.name, sep = ',', index = include_index)
    cur = connection.cursor()
    sql = "copy " + destination_table + columns + " from stdin delimiter ',' skip 1"
    
    if verbose == True:
        print(sql)
    
    with open(tmp_file.name, "rb") as fs:
        cur.copy(sql, fs)
        connection.commit()
    tmp_file.close()
    
    return str(len(source_df)) + ' row(s) written to table ' + destination_table


copy_to_vertica(predictions, 'analytics.model_scoring_predictions', conn)

Number of users: 198


'198 row(s) written to table analytics.model_scoring_predictions'

In [18]:
query = "SELECT * FROM analytics.model_scoring_predictions LIMIT 10"

In [19]:
utils.sql_query(cur, query)

ConnectionError: 

In [37]:
# predictions.to_csv("scoring/clv_prediction_" + users_from + "-" + users_till, index=False)

### LOAD RESULTS INTO VERTICA

In [ ]:
# print("Scoring loaded to vertica")

In [ ]:
# when running as a script
if __name__ == "__main__":
    users_from = sys.argv[1]
    users_till = sys.argv[2]
    main(users_from, users_till)